In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm, trange

In [2]:
# from datasets import load_dataset
# dataset = load_dataset('breadlicker45/youtube-comments-180k', split='train')

In [3]:
# dataset.to_csv('youtube-comments-180k.csv')

In [4]:
yt_df = pd.read_csv("youtube-comments-180k.csv")
yt_df.info()
yt_df = yt_df[yt_df['text'].str.len()<300] #character limit = 300
yt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    187480 non-null  object
dtypes: object(1)
memory usage: 1.4+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 164699 entries, 0 to 187481
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    164699 non-null  object
dtypes: object(1)
memory usage: 2.5+ MB


In [5]:
# train, test = train_test_split(yt_df, test_size=0.35)
# train.info()
# test.info()
# train.to_csv('train_yt.csv', index = False)
# test.to_csv('test_yt.csv', index = False)
test = pd.read_csv("test_yt.csv")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57645 entries, 0 to 57644
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    57645 non-null  object
dtypes: object(1)
memory usage: 450.5+ KB


# Sentiment Analysis using RoBerta

In [6]:
#setup pre-trained roberta model from https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest 
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# Run for Roberta Model
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

print(polarity_scores_roberta("I am happy"))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'roberta_neg': 0.0077943704, 'roberta_neu': 0.030709133, 'roberta_pos': 0.96149653}


In [7]:
# Run Roberta on test data to get analysis prior leet substitution
test1_result = {'text':[], 'Neg':[], 'Neu':[], 'Pos':[]}

for i in trange(len(test['text'].values)):
    test1_result['text'].append(test['text'].values[i])

    polarity_scores = polarity_scores_roberta(test['text'].values[i])
    test1_result['Neg'].append(polarity_scores['roberta_neg'])
    test1_result['Neu'].append(polarity_scores['roberta_neu'])
    test1_result['Pos'].append(polarity_scores['roberta_pos'])

test1_result_df = pd.DataFrame(test1_result)
test1_result_df.head()

100%|██████████| 57645/57645 [1:07:49<00:00, 14.17it/s]


,text,Neg,Neu,Pos
0,Ok I hate to tell everyone but 90s quality is ...,0.171523,0.257676,0.570801
1,Actually the recent windows update made my pc ...,0.014980,0.076991,0.908029
2,Tom I really hope you enjoyed your stay in War...,0.002816,0.012929,0.984254
3,"OR, more likely, they hired a professional who...",0.330295,0.606575,0.063130
4,I am confused because of Linus's tshirt.,0.527813,0.462156,0.010031


In [ ]:
test1_result_df.to_csv("test1_noLeet_setiment_result.csv", index = False)